In [ ]:
import pandas as pd
import polars as pl
import numpy as np
import networkx as nx
import json
import time
import sys
sys.path.insert(1, '../../framework')
from racetrack import *
rt = RACETrack()
df = pd.read_csv('llama3_8b_2014_vast_sbs.csv')
len(df), df['llama3_8b_time'].sum()/60.0/60.0 # appears to be about 8 hours...

In [ ]:
_failed_parse_counts_, _failed_parses_, _successful_parse_counts_ = 0, [], 0
for _json_str_ in df['llama3_8b_response']:
    if '{' not in _json_str_ or '}' not in _json_str_: 
        _failed_parse_counts_ += 1
        _failed_parses_.append(_json_str_)
        continue
    _json_str_ = _json_str_[_json_str_.index('{'):_json_str_.rindex('}')+1]
    try:
        json.loads(_json_str_)
        _successful_parse_counts_ += 1
    except:
        try:
            json.loads(_json_str_ + '}')
            _successful_parse_counts_ += 1
        except:
            try:
                json.loads(_json_str_ + '] }')
                _successful_parse_counts_ += 1
            except:    
                _failed_parse_counts_ += 1
                _failed_parses_.append(_json_str_)
_successful_parse_counts_ ,_failed_parse_counts_

In [ ]:
j = 3
_splits_ = _failed_parses_[j].split('\n')
for i in range(len(_splits_)):
    print(f'{i:-3} {_splits_[i]}')
json.loads(_failed_parses_[j] + '] }')